In [10]:
from google.colab import files
upload = files.upload()

Saving mnist_dataset.csv to mnist_dataset.csv


In [11]:
#importing necessary libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
import pandas as pd
df = pd.read_csv("mnist_dataset.csv")

In [13]:
df.head()

,label,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,...,pixel_774,pixel_775,pixel_776,pixel_777,pixel_778,pixel_779,pixel_780,pixel_781,pixel_782,pixel_783
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
import numpy as np

def softmax(z):
    """
    Compute the softmax probabilities for a given input matrix.

    Parameters:
    z (numpy.ndarray): Logits (raw scores) of shape (m, n), where
        - m is the number of samples.
        - n is the number of classes.

    Returns:
    numpy.ndarray: Softmax probability matrix of shape (m, n), where
        each row sums to 1 and represents the probability
        distribution over classes.

    Notes:
    - The input to softmax is typically computed as: z = XW + b.
    - Uses numerical stabilization by subtracting the max value per row.
    """
    z = z - np.max(z, axis=1, keepdims=True)  # Prevents large values
    exp_z = np.exp(z)  # Apply exponent
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)  # Normalize

In [16]:
# This test case checks that each row in the resulting softmax probabilities sums to 1, which is the

# Example test case



z_test = np.array([[2.0, 1.0, 0.1], [1.0, 1.0, 1.0]])
softmax_output = softmax(z_test)
# Verify if the sum of probabilities for each row is 1 using assert
row_sums = np.sum(softmax_output, axis=1)
# Assert that the sum of each row is 1
assert np.allclose(row_sums, 1), f"Test failed: Row sums are {row_sums}"
print("Softmax function passed the test case!")

Softmax function passed the test case!


In [17]:
import numpy as np

def predict_softmax(X, W, b):
    """
    Predict class labels using a trained softmax model.

    Steps:
    1. Compute raw scores (logits) using the formula: z = XW + b.
    2. Apply softmax to turn raw scores into probabilities.
    3. Pick the class with the highest probability for each sample.

    Parameters:
    X (numpy.ndarray): Feature matrix (n, d), where:
        - n = number of samples (rows)
        - d = number of features (columns)
    W (numpy.ndarray): Weight matrix (d, c), where:
        - c = number of classes
    b (numpy.ndarray): Bias vector (c,).

    Returns:
    numpy.ndarray: Predicted class labels (n,), each value is a class index.
    """

    z = np.dot(X, W) + b


    z = z - np.max(z, axis=1, keepdims=True)
    exp_z = np.exp(z)
    softmax_probs = exp_z / np.sum(exp_z, axis=1, keepdims=True)


    predicted_classes = np.argmax(softmax_probs, axis=1)

    return predicted_classes

In [18]:
# The test function ensures that the predicted class labels have the same number of elements as the

# Define test case
X_test = np.array([[0.2, 0.8], [0.5, 0.5], [0.9, 0.1]]) # Feature matrix (3 samples, 2 features)
W_test = np.array([[0.4, 0.2, 0.1], [0.3, 0.7, 0.5]]) # Weights (2 features, 3 classes)
b_test = np.array([0.1, 0.2, 0.3]) # Bias (3 classes)
# Expected Output:
# The function should return an array with class labels (0, 1, or 2)
y_pred_test = predict_softmax(X_test, W_test, b_test)
# Validate output shape
assert y_pred_test.shape == (3,), f"Test failed: Expected shape (3,), got {y_pred_test.shape}"
# Print the predicted labels
print("Predicted class labels:", y_pred_test)

Predicted class labels: [1 1 0]


In [19]:
def loss_softmax(y_pred,y):
 """
 Computethecross-entropylossforasinglesample.
 Parameters:
 y_pred(numpy.ndarray):Predictedprobabilitiesofshape(c,)forasinglesample,
 wherecisthenumberofclasses.
 y (numpy.ndarray):Truelabels(one-hotencoded)ofshape(c,),wherecisthenumber ofclasses.
 Returns:
 float:Cross-entropylossforthegivensample.
 """
 loss = np.sum(y* np.log(y_pred))
 return loss

In [20]:
import numpy as np

# Softmax function to convert logits to probabilities
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))  # Stabilize exponentiation
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

# Cross-entropy loss function
def loss_softmax(y_pred, y_true):
    # Compute the softmax probabilities for the predicted values
    y_pred_softmax = softmax(y_pred)

    # Calculate cross-entropy loss (negative log likelihood)
    # Clip the predicted probabilities to avoid log(0)
    epsilon = 1e-15
    y_pred_softmax = np.clip(y_pred_softmax, epsilon, 1. - epsilon)

    # Cross-entropy loss formula
    loss = -np.sum(y_true * np.log(y_pred_softmax)) / y_true.shape[0]

    return loss

# Define correct predictions (low loss scenario)
y_true_correct = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])  # True one-hot labels
y_pred_correct = np.array([[0.9, 0.05, 0.05],
                           [0.1, 0.85, 0.05],
                           [0.05, 0.1, 0.85]])  # High confidence in the correct class

# Define incorrect predictions (high loss scenario)
y_pred_incorrect = np.array([[0.05, 0.05, 0.9],  # Highly confident in the wrong class
                             [0.1, 0.05, 0.85],
                             [0.85, 0.1, 0.05]])

# Compute loss for both cases
loss_correct = loss_softmax(y_pred_correct, y_true_correct)
loss_incorrect = loss_softmax(y_pred_incorrect, y_true_correct)

# Validate that incorrect predictions lead to a higher loss
assert loss_correct < loss_incorrect, f"Test failed: Expected loss_correct < loss_incorrect, but got {loss_correct:.4f} >= {loss_incorrect:.4f}"

# Print results
print(f"Cross-Entropy Loss (Correct Predictions): {loss_correct:.4f}")
print(f"Cross-Entropy Loss (Incorrect Predictions): {loss_incorrect:.4f}")

Cross-Entropy Loss (Correct Predictions): 0.6414
Cross-Entropy Loss (Incorrect Predictions): 1.4581


In [25]:
def cost_softmax(X, y, W, b):
    """
    Compute the average softmax regression cost (cross-entropy loss) over all samples using loops.

    Parameters:
    X (numpy.ndarray): Feature matrix of shape (n, d), where n is the number of samples and d is the number of features.
    y (numpy.ndarray): True labels (one-hot encoded) of shape (n, c), where n is the number of samples and c is the number of classes.
    W (numpy.ndarray): Weight matrix of shape (d, c).
    b (numpy.ndarray): Bias vector of shape (c,).

    Returns:
    float: Average softmax cost (cross-entropy loss) over all samples.
    """
    n, d = X.shape
    c = y.shape[1]
    total_loss = 0.0

    for i in range(n):
        logits = np.dot(X[i], W) + b
        exp_logits = np.exp(logits - np.max(logits))
        probs = exp_logits / np.sum(exp_logits)
        sample_loss = -np.sum(y[i] * np.log(probs + 1e-9))
        total_loss += sample_loss

    return total_loss / n

In [26]:
# The test case assures that the cost for the incorrect prediction should be higher than for the correct prediction, confirming that the cost function behaves as expected.
import numpy as np
# Example 1: Correct Prediction (Closer predictions)
X_correct = np.array([[1.0, 0.0], [0.0, 1.0]]) # Feature matrix for correct predictions
y_correct = np.array([[1, 0], [0, 1]]) # True labels (one-hot encoded, matching predictions)
W_correct = np.array([[5.0, -2.0], [-3.0, 5.0]]) # Weights for correct prediction
b_correct = np.array([0.1, 0.1]) # Bias for correct prediction
# Example 2: Incorrect Prediction (Far off predictions)
X_incorrect = np.array([[0.1, 0.9], [0.8, 0.2]]) # Feature matrix for incorrect predictions
y_incorrect = np.array([[1, 0], [0, 1]]) # True labels (one-hot encoded, incorrect predictions)
W_incorrect = np.array([[0.1, 2.0], [1.5, 0.3]]) # Weights for incorrect prediction
b_incorrect = np.array([0.5, 0.6]) # Bias for incorrect prediction
# Compute cost for correct predictions
cost_correct = cost_softmax(X_correct, y_correct, W_correct, b_correct)
# Compute cost for incorrect predictions
cost_incorrect = cost_softmax(X_incorrect, y_incorrect, W_incorrect, b_incorrect)
# Check if the cost for incorrect predictions is greater than for correct predictions
assert cost_incorrect > cost_correct, f"Test failed: Incorrect cost {cost_incorrect} is not greater than correct cost {cost_correct}"
# Print the costs for verification
print("Cost for correct prediction:", cost_correct)
print("Cost for incorrect prediction:", cost_incorrect)
print("Test passed!")

Cost for correct prediction: 0.0006234354127112888
Cost for incorrect prediction: 0.2993086122417495
Test passed!


In [27]:
import numpy as np

def cost_softmax(X, y, W, b):
    """
    Compute the average softmax regression cost (cross-entropy loss) over all samples using a for loop.

    Parameters:
    X (numpy.ndarray): Feature matrix of shape (n, d), where n is the number of samples and d is the
    number of features.
    y (numpy.ndarray): True labels (one-hot encoded) of shape (n, c), where n is the number of
    samples and c is the number of classes.
    W (numpy.ndarray): Weight matrix of shape (d, c).
    b (numpy.ndarray): Bias vector of shape (c,).

    Returns:
    float: Average softmax cost (cross-entropy loss) over all samples.
    """
    n = X.shape[0]  # Number of samples
    total_loss = 0.0  # Initialize total loss
    for i in range(n):
        y_pred = softmax(np.dot(X[i], W) + b)  # Compute softmax probabilities

    return total_loss / n  # Return average loss

In [28]:
def cost_softmax(X, y, W, b):
    """
    Compute the average softmax regression cost (cross-entropy loss) over all samples.

    Parameters:
    X (numpy.ndarray): Feature matrix of shape (n, d).
    y (numpy.ndarray): True labels (one-hot encoded) of shape (n, c).
    W (numpy.ndarray): Weight matrix of shape (d, c).
    b (numpy.ndarray): Bias vector of shape (c,).

    Returns:
    float: Average softmax cost (cross-entropy loss) over all samples.
    """
    # Compute logits (raw scores)
    logits = np.dot(X, W) + b

    # Use previously defined softmax function
    probs = softmax(logits)

    # Compute loss using vectorized approach
    total_loss = -np.sum(y * np.log(probs + 1e-9))  # Adding small value for numerical stability

    return total_loss / X.shape[0]  # Average loss over all samples
# The test case assures that the cost for the incorrect prediction should be higher than for the

import numpy as np
# Example 1: Correct Prediction (Closer predictions)
X_correct = np.array([[1.0, 0.0], [0.0, 1.0]]) # Feature matrix for correct predictions
y_correct = np.array([[1, 0], [0, 1]]) # True labels (one-hot encoded, matching predictions)
W_correct = np.array([[5.0, -2.0], [-3.0, 5.0]]) # Weights for correct prediction
b_correct = np.array([0.1, 0.1]) # Bias for correct prediction
# Example 2: Incorrect Prediction (Far off predictions)
X_incorrect = np.array([[0.1, 0.9], [0.8, 0.2]]) # Feature matrix for incorrect predictions
y_incorrect = np.array([[1, 0], [0, 1]]) # True labels (one-hot encoded, incorrect predictions)
W_incorrect = np.array([[0.1, 2.0], [1.5, 0.3]]) # Weights for incorrect prediction
b_incorrect = np.array([0.5, 0.6]) # Bias for incorrect prediction
# Compute cost for correct predictions
cost_correct = cost_softmax(X_correct, y_correct, W_correct, b_correct)
# Compute cost for incorrect predictions
cost_incorrect = cost_softmax(X_incorrect, y_incorrect, W_incorrect, b_incorrect)
# Check if the cost for incorrect predictions is greater than for correct predictions
assert cost_incorrect > cost_correct, f"Test failed: Incorrect cost {cost_incorrect} is not greater than correct cost {cost_correct}"
# Print the costs for verification
print("Cost for correct prediction:", cost_correct)
print("Cost for incorrect prediction:", cost_incorrect)
print("Test passed!")


Cost for correct prediction: 0.0006234354127112888
Cost for incorrect prediction: 0.2993086122417495
Test passed!
